In [147]:
import pandas as pd
import plotly.express as px
import os

In [148]:
pd.read_pickle("failed_experiments.pkl").columns

Index([], dtype='object')

In [149]:
def make_nice(df):
    return df.rename(lambda x: x.replace("_", " ").title(), axis=1)

In [153]:
import pickle
if os.path.getsize("results.pkl") == 0: print("ah")
else: 
    with open("results.pkl", 'rb') as f:
        unpickler = pickle.Unpickler(f)
        df = unpickler.load()
    df

ah


In [150]:
# df = pd.read_pickle("hpc_first_results.pkl")
df = pd.read_pickle("results.pkl")
sorted(make_nice(df).columns)

EOFError: Ran out of input

In [ ]:

df["nodes"] = df["nodes"].astype(str)
df["experiment_type"] = df["experiment_type"].map(lambda x: "Single Pairs" if x == "Single" else x)
df["perturber_class"] = df["perturber_class"].map(lambda x: "GreedyMin" if x == "MinFirst" else x)


In [ ]:
print(len(df), sum(df["Success"]), sum(df["Success"])/len(df))
for experiment_type in df["experiment_type"].unique():
    fig = px.histogram(df[df["experiment_type"] == experiment_type], title=f"{experiment_type} - Success Rate", x="perturber_class", color="Status",hover_data=["LP Status"], category_orders={"perturber_class": ["PathAttack", "GreedyFirst", "GreedyMin"]})
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.show()

In [ ]:
df["Average Add Time"] = df["Add Times"].map(lambda x: sum(x) / len(x) if len(x)>0 else None)
df["Average Perturb Time"] = df["Perturb Times"].map(lambda x: sum(x) / len(x) if len(x)>0 else None)
df["Total Add Time"] = df["Add Times"].map(sum)
df["Total Perturb Time"] = df["Perturb Times"].map(sum)
df["Overhead Time"] = df["Time Taken"] - df["Total Add Time"] - df["Total Perturb Time"]

In [ ]:
# df.to_csv("results.csv")

In [ ]:
index_cols = ["nodes", "condition_index", "configuration_index"]
global_cols = ["epsilon", "n_nodes_per_experiment", "n_experiments", "n_trials", "min_path_length", "use_multithreading", "global_budget"]
extra_cols = ["Status","Add Times", "Perturb Times", "Perturbation Dict","source", "target", "LP Status", "IIS_paths", "IIS_edges", "IIS_global_budget", "Supporting Paths", "Global Budget Slack"]
config_cols = ["perturber_class", "graph_name", "k", "weights", "experiment_type", "local_budget"]
data_cols = ["Time Taken", "Iterations", "Number of Paths", "Number of Edges", "Original Distance", "Final Distance", "Success", "Total Perturbations","Total Add Time", "Total Perturb Time", "Overhead Time"]

In [ ]:
# Average Over Trials
a = df[index_cols+data_cols].groupby(index_cols).mean()
a = a.join(df.set_index(index_cols)[config_cols]) # Add back config information
a = a.drop_duplicates()
len(a)

In [ ]:
## TODO: Instances where the LP is infeasible or runs out of iterations should be removed from the data

for y_val in ["Time Taken", "Total Perturbations"]:
    for experiment_type in a["experiment_type"].unique():
        for i, weights in enumerate(a["weights"].unique()):
            for j, graph_name in enumerate(a["graph_name"].unique()):
                s = a[(a["experiment_type"] == experiment_type) & (a["k"] == 5) & (a["graph_name"] == graph_name) & (a["weights"] == weights)]
                fig = px.box(s, 
                            title="",#f"{experiment_type}: {y_val}",
                            # x="nodes", 
                            y=y_val, 
                            boxmode="group", 
                            color="perturber_class",
                            color_discrete_map={
                                "PathAttack": '#636EFA', # blue
                                "GreedyFirst": '#EF553B', # red
                                "GreedyMin": '#00CC96', # green
                            },
                            height=400,
                            width=400,
                            hover_data=data_cols,
                            category_orders={"Status": ["Success"]+[s for s in df["Status"].unique() if s != "Success"],
                                            "Success": [True, False]},
                            # points="all",
                            # boxpoints=False,
                            )
                fig.update_xaxes(visible=False, matches=None)
                if i!=0 or j!= 0: fig.update_layout(showlegend=False)
                if j!=0: fig.update_layout(yaxis=dict(visible=False))

                fig.update_layout(margin=dict(l=10, r=10, t=20, b=10), legend_title_text='Perturber Class')
                fig.show()
                image_path = f"Results/Images/{experiment_type}/{graph_name}/"
                if not os.path.exists(os.path.dirname(image_path)):
                    os.makedirs(os.path.dirname(image_path))
                fig.write_image(image_path+f"{weights} weights - {y_val}.png")

In [ ]:
## TODO: Instances where the LP is infeasible or runs out of iterations should be removed from the data

for y_val in ["Time Taken", "Total Perturbations"]:
    for experiment_type in a["experiment_type"].unique():
        if a.empty: continue
        fig = px.box(a[(a["experiment_type"] == experiment_type) & (a["k"] == 5)], 
                    title="",#f"{experiment_type}: {y_val}",
                    # x="nodes", 
                    y=y_val, 
                    boxmode="group", 
                    color="perturber_class", 
                    color_discrete_map={
                        "PathAttack": '#636EFA', # blue
                        "GreedyFirst": '#EF553B', # red
                        "GreedyMin": '#00CC96', # green
                    },
                    facet_col="weights", 
                    facet_row="graph_name",  
                    height=300,
                    width=1000,
                    hover_data=data_cols,
                    )
        fig.update_xaxes(visible=False, matches=None)
        fig.for_each_annotation(lambda a: a.update(text=': '.join(a.text.replace("_"," ").title().split("="))))
        fig.update_layout(
    margin=dict(l=10, r=10, t=20, b=10), legend_title_text='Perturber Class'
    )
        fig.show()
        fig.write_image(f"Results/{experiment_type} - {y_val}.png")

In [ ]:
# b = a.reset_index().set_index(["configuration_index", "condition_index"])
# for experiment_type in a.experiment_type.unique():
#     fig = px.bar(b[b["experiment_type"]==experiment_type], 
#                 title=f"{experiment_type} Results",
#                 x="nodes", 
#                 y="Time Taken", 
#                 barmode="group", 
#                 color="perturber_class", 
#                 facet_col="weights", 
#                 facet_row="k", 
#                 pattern_shape="Success", 
#                 # height=1000,
#                 # hover_data=["top_k", "Average Add Time", "Average Perturb Time", "Original Distance", "Number of Paths", "Number of Edges", "Iterations", 'Final Distance',"Status", "LP Status", "nodes"],
#                 category_orders={"experiment_type": ["Single", "Sets", "Multiple Pairs"], 
#                                 "Status": ["Success"]+[s for s in df["Status"].unique() if s != "Success"],
#                                 "Success": [True, False]}
#                 )
#     fig.update_xaxes(visible=False)
#     fig.update_xaxes(matches=None)
#     # fig.update_xaxes(visible=True)
#     fig.show()

In [ ]:
c = a.reset_index().set_index(["condition_index", "nodes"]+config_cols[1:]).groupby("perturber_class")
optimal = c.get_group("PathAttack").reset_index().set_index(["condition_index", "nodes"]+config_cols[1:])
mean_dfs = dict()
for y_val in ["Total Perturbations", "Time Taken", "Iterations", "Number of Paths"]:
        mean_df = pd.DataFrame()
        for name, group in c:
            mean_df[name] = group.reset_index().set_index(["condition_index", "nodes"]+config_cols[1:])[y_val] / optimal[y_val]
        mean_df = mean_df.reset_index().groupby(["experiment_type", "graph_name", "weights", "k"]).mean().round(3)
        mean_df.drop(columns=["condition_index"], inplace=True)
        mean_dfs[y_val] = mean_df
        for experiment_type in df.experiment_type.unique():
            with open(f"Results/Relative {y_val} Mean Results - {experiment_type}.tex", "w") as f:
                f.write(mean_df.loc[experiment_type].to_latex())

In [ ]:
mean_dfs["Iterations"]